In [ ]:
! pip install torchtext==0.10.1 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 831.4/831.4 MB 1.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1+cu116
    Uninstalling torch-1.13.1+cu116:
      Successfully uninstalled torch-1.13.1+cu116
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.14.1
    Uninstalling torchtext-0.14.1:
      Successfully uninstalled torchtext-0.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.14.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.
torchaudio 0.13.1+cu116 requires torch==1.13.1, but you have torch 1.9.1 which is incompatible.


# Bidirectional RNN


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


We can see how many examples are in each split by checking their length.

In [ ]:
training_fields = [(None, None), ('refinedText',TEXT), (None, None), (None, None), ('tags', LABEL)]

In [ ]:
training_data=data.TabularDataset(path = 'output-labelled.csv',format = 'csv',fields = training_fields,skip_header = True)

In [ ]:
training_data.examples[1].tags

'0'

In [ ]:
valid_data = data.TabularDataset(path = 'valid_input_for_neural_not_refined.csv',format = 'csv',fields = training_fields,skip_header = True)

In [ ]:
import random
train_data, test_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [ ]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "fasttext.simple.300d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

.vector_cache/wiki.simple.vec: 293MB [00:01, 171MB/s]                           
100%|██████████| 111051/111051 [00:09<00:00, 12038.36it/s]


Now, let's check unique tokens in TEXT and LABEL vocabulary.


In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 6984
Unique tokens in LABEL vocabulary: 3


In [ ]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 507), ('to', 439), ("'s", 432), ('The', 375), ('a', 363), ('?', 327), ('This', 298), ('You', 271), ('of', 260), ('in', 251), ('is', 249), ('"', 245), ('you', 239), ('-', 230), (',', 210), ('.', 199), ("'", 190), ('for', 185), ('Is', 175), (':', 164)]


In [ ]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', 'to', "'s", 'The', 'a', '?', 'This', 'You']


In [ ]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.refinedText),
    sort_within_batch = False,
    device = device)

We are working with  cuda


## Project Task 2 - Bidirectional Multi-Layer RNN




In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):

        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.rnn = nn.RNN(input_size = embedding_dim, 
                          hidden_size = hidden_dim, 
                          num_layers = n_layers,
                          bidirectional = bidirectional, 
                          dropout = dropout,
                          batch_first = False)

        self.fc = nn.Linear(hidden_dim * n_layers, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        emb = self.dropout(self.embedding(text))

       
        packed_sequences = nn.utils.rnn.pack_padded_sequence(emb, 
                                                             lengths = text_lengths.cpu(), 
                                                             batch_first = False, 
                                                             enforce_sorted = False) 
        _, hidden = self.rnn(packed_sequences)

        
        hidden = self.dropout(torch.cat([hidden[-2,:,:], hidden[-1,:,:]], dim = 1))

        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
HIDDEN_DIM = 256
OUTPUT_DIM = 3
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,776,675 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([6984, 300])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [ 0.0104, -0.1829,  0.0761,  ..., -0.1362, -0.2240, -0.0552],
        ...,
        [-0.0745,  0.6966, -1.0372,  ..., -1.6579,  1.1719,  0.7859],
        [-1.8401, -2.8015, -1.2543,  ..., -0.1950, -0.4740, -0.9836],
        [ 0.0208, -0.5172, -0.9634,  ..., -0.1467,  0.9068,  0.5689]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0104, -0.1829,  0.0761,  ..., -0.1362, -0.2240, -0.0552],
        ...,
        [-0.0745,  0.6966, -1.0372,  ..., -1.6579,  1.1719,  0.7859],
        [-1.8401, -2.8015, -1.2543,  ..., -0.1950, -0.4740, -0.9836],
        [ 0.0208, -0.5172, -0.9634,  ..., -0.1467,  0.9068,  0.5689]])


## Train the Model

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    # rounded_preds = torch.round(torch.sigmoid(preds))
    # correct = (rounded_preds == y).float() #convert into float for division 
    # acc = correct.sum() / len(correct)
    # print(torch.argmax(preds.softmax(1), dim = 1))
    # print(y)

    acc = (torch.argmax(preds.softmax(1), dim = 1) == y).sum()
    # print(acc)
    # print(len(y))
    return torch.tensor(acc/len(y))

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.refinedText
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, torch.nn.functional.one_hot(batch.tags.to(torch.int64), 3).float())
        
        acc = binary_accuracy(predictions, batch.tags)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.refinedText
            
            predictions = model(text, text_lengths)

            loss = criterion(predictions, torch.nn.functional.one_hot(batch.tags.to(torch.int64), 3).float())
        
            acc = binary_accuracy(predictions, batch.tags)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(data['review'], data['condition'],
                                                    stratify=data['condition'], 
                                                    test_size=0.25)

In [ ]:
torch.save(model,'bilstm_model')

In [ ]:
N_EPOCHS = 5
path = F"/content/gdrive/My Drive/birnn_model.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

<ipython-input-23-7ceeb402aa47>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(acc/len(y))


Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.633 | Train Acc: 44.38%
	 Val. Loss: 0.613 |  Val. Acc: 47.36%
Epoch: 02 | Epoch Time: 0m 1s
	Train Loss: 0.576 | Train Acc: 52.63%
	 Val. Loss: 0.622 |  Val. Acc: 48.20%
Epoch: 03 | Epoch Time: 0m 1s
	Train Loss: 0.548 | Train Acc: 57.77%
	 Val. Loss: 0.621 |  Val. Acc: 51.32%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.484 | Train Acc: 65.36%
	 Val. Loss: 0.655 |  Val. Acc: 51.44%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.438 | Train Acc: 70.76%
	 Val. Loss: 0.712 |  Val. Acc: 47.24%


In [ ]:
model.load_state_dict(torch.load(path, map_location=device))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.594 | Test Acc: 51.15%


<ipython-input-23-7ceeb402aa47>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(acc/len(y))


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    # prediction = torch.sigmoid(model(tensor, length_tensor))
    return (model(tensor, length_tensor).softmax(1)).argmax(1)

An example negative review...

In [ ]:
predict_sentiment(model, "six ways to solve this puzzle")

tensor([0], device='cuda:0')

An example positive review...

In [ ]:
predict_sentiment(model, "these are the 78 ways")

tensor([0], device='cuda:0')